In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 

from tensorflow.keras import datasets, layers, models, regularizers, initializers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.io import imshow
from skimage.transform import rotate
from skimage.filters.edges import convolve

from skimage.io import imshow, imread
from skimage.transform import resize
from skimage.color import rgb2gray

np.random.seed(42)

In [2]:
df = pd.read_csv('data/labels_df.csv')

loaded = np.load('data/image_array.npz')
all_img = loaded['a']
bw_img = loaded['b']

labels = df.masterCategory.copy()
labels = labels.astype('category')
labels = labels.cat.codes

In [3]:
masterCat_codes = {}
masterCat_counts = {}
for i in range(len(labels.value_counts().index)):
    s = labels.value_counts().index[i]
    t = df.masterCategory.value_counts().index[i]
    u = labels.value_counts().values[i]
    masterCat_codes[s] = t
    masterCat_counts[t] = u

print(masterCat_counts, '\n', '\n', masterCat_codes)

{'Apparel': 21341, 'Accessories': 11004, 'Footwear': 9119, 'Personal Care': 2393, 'Free Items': 104, 'Sporting Goods': 25, 'Home': 1} 
 
 {1: 'Apparel', 0: 'Accessories', 2: 'Footwear', 5: 'Personal Care', 3: 'Free Items', 6: 'Sporting Goods', 4: 'Home'}


In [4]:
#shuffle images and target var
shuffle = np.random.choice( np.arange(len(bw_img)), size=len(bw_img), replace=False)
X = bw_img[shuffle].reshape(-1,80,60,1)
y = labels.values[shuffle]

#at end, use this to view images in color
colorX = [all_img[ix] for ix in shuffle]

n_train = round(len(X)*.70)

X_train = X[:n_train]
y_train = y[:n_train]
shuffle_train = shuffle[:n_train]

X_test = X[n_train:]
y_test = y[n_train:]
shuffle_test = shuffle[n_train:]

In [5]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [6]:
y_train.shape

(30791, 7)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

In [8]:
model = models.Sequential()

#model.add(layers.MaxPooling2D((2, 2)))

In [9]:
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))

W0723 23:14:51.438689 139900462184256 deprecation.py:506] From /home/ubuntu/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 78, 58, 32)        320       
Total params: 320
Trainable params: 320
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.add(Flatten())
model.add(Dense(7, activation='softmax'))

In [12]:
# model.add(layers.Flatten())
# model.add(layers.Dense(1, activation='sigmoid',
#                       kernel_regularizer=regularizers.l2(0.003)))

In [13]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 30791 samples, validate on 13196 samples
Epoch 1/10
30791/30791 [==============================] - 21s 696us/sample - loss: 57.2570 - acc: 0.8959 - val_loss: 0.9676 - val_acc: 0.9248
Epoch 2/10
30791/30791 [==============================] - 18s 585us/sample - loss: 0.4090 - acc: 0.9520 - val_loss: 0.9042 - val_acc: 0.9212
Epoch 3/10
30791/30791 [==============================] - 18s 572us/sample - loss: 0.2234 - acc: 0.9681 - val_loss: 0.9717 - val_acc: 0.9412
Epoch 4/10
30791/30791 [==============================] - 18s 584us/sample - loss: 0.2032 - acc: 0.9719 - val_loss: 1.0121 - val_acc: 0.9399
Epoch 5/10
30791/30791 [==============================] - 18s 593us/sample - loss: 0.1910 - acc: 0.9746 - val_loss: 1.0616 - val_acc: 0.9389
Epoch 6/10
30791/30791 [==============================] - 18s 598us/sample - loss: 0.1496 - acc: 0.9778 - val_loss: 0.9863 - val_acc: 0.9424
Epoch 7/10
30791/30791 [==============================] - 18s 591us/sample - loss: 0.1817 - acc: 0.9692

In [15]:
# get the predictions for the test data
predicted_classes = model.predict_classes(X_test)

# get the indices to be plotted
y_true = y[n_train:]
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]

In [16]:
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(6)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.84      0.89      0.87      3239
     Class 1       0.95      0.94      0.95      6496
     Class 2       0.97      0.96      0.96      2733
     Class 3       0.00      0.00      0.00        31
     Class 4       0.82      0.76      0.79       691
     Class 5       0.33      0.17      0.22         6

    accuracy                           0.92     13196
   macro avg       0.65      0.62      0.63     13196
weighted avg       0.92      0.92      0.92     13196



In [17]:
from imblearn.over_sampling import SMOTE

In [45]:
X_train.shape, y_train.shape

((30791, 80, 60, 1), (30791, 7))

In [18]:
smote = SMOTE('minority')

In [20]:
X_smo, y_smo = smote.fit_sample(X_train.reshape(-1,4800), y_train)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1, n_neighbors = 6

In [23]:
from sklearn.utils import class_weight